In [ ]:
# 1️⃣ Importation des librairies
import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv

# 2️⃣ Chargement du dataset Cora
dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0]

print("Nombre de nœuds:", data.num_nodes)
print("Nombre de features par nœud:", dataset.num_node_features)
print("Nombre de classes:", dataset.num_classes)

# 3️⃣ Définition du modèle GCN
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

model = GCN(in_channels=dataset.num_node_features,
            hidden_channels=16,
            out_channels=dataset.num_classes)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

# 4️⃣ Entraînement du modèle
def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

def test():
    model.eval()
    out = model(data.x, data.edge_index)
    pred = out.argmax(dim=1)

    accs = []
    for mask in [data.train_mask, data.val_mask, data.test_mask]:
        correct = pred[mask].eq(data.y[mask]).sum().item()
        acc = correct / mask.sum().item()
        accs.append(acc)
    return accs  # [train_acc, val_acc, test_acc]

# Boucle d'entraînement
best_val_acc = 0
best_test_acc = 0
for epoch in range(1, 201):
    loss = train()
    train_acc, val_acc, test_acc = test()
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_test_acc = test_acc
    if epoch % 20 == 0:
        print(f"Epoch: {epoch:03d}, Loss: {loss:.4f}, Train: {train_acc:.4f}, Val: {val_acc:.4f}, Test: {test_acc:.4f}")

print(f"\n✅ Meilleure performance sur le test set: {best_test_acc:.4f}")


Processing...
Done!


Nombre de nœuds: 2708
Nombre de features par nœud: 1433
Nombre de classes: 7
Epoch: 020, Loss: 0.3065, Train: 1.0000, Val: 0.7920, Test: 0.8040
Epoch: 040, Loss: 0.0717, Train: 1.0000, Val: 0.7760, Test: 0.8030
Epoch: 060, Loss: 0.0320, Train: 1.0000, Val: 0.7680, Test: 0.7930
Epoch: 080, Loss: 0.0465, Train: 1.0000, Val: 0.7800, Test: 0.8060
Epoch: 100, Loss: 0.0397, Train: 1.0000, Val: 0.7780, Test: 0.8040
Epoch: 120, Loss: 0.0465, Train: 1.0000, Val: 0.7800, Test: 0.8140
Epoch: 140, Loss: 0.0232, Train: 1.0000, Val: 0.7720, Test: 0.8110
Epoch: 160, Loss: 0.0230, Train: 1.0000, Val: 0.7780, Test: 0.8140
Epoch: 180, Loss: 0.0221, Train: 1.0000, Val: 0.7700, Test: 0.7980
Epoch: 200, Loss: 0.0332, Train: 1.0000, Val: 0.7680, Test: 0.8090

✅ Meilleure performance sur le test set: 0.8040


In [ ]:
# ============================================
# 1️⃣ Importation des librairies
# ============================================
import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GATConv
import copy


# Vérifier CUDA
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Device utilisé :", device)


# ============================================
# 2️⃣ Chargement du dataset Cora
# ============================================
dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0].to(device)


print("Nombre de nœuds:", data.num_nodes)
print("Nombre de features par nœud:", dataset.num_node_features)
print("Nombre de classes:", dataset.num_classes)


# ============================================
# 3️⃣ Définition du modèle SuperGAT optimisé
# ============================================
class SuperGAT(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, heads=8):
        super().__init__()
        self.dropout_feat = torch.nn.Dropout(0.6)

        # Attention dropout = 0.6
        self.conv1 = GATConv(in_channels, hidden_channels, heads=heads,
                             dropout=0.6)

        # 8 heads en sortie pour meilleure stabilité !
        self.conv2 = GATConv(hidden_channels * heads, out_channels,
                             heads=8, concat=False, dropout=0.6)

    def forward(self, x, edge_index):
        x = self.dropout_feat(x)
        x = self.conv1(x, edge_index)
        x = F.leaky_relu(x, negative_slope=0.2)
        x = self.dropout_feat(x)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)


model = SuperGAT(
    in_channels=dataset.num_node_features,
    hidden_channels=8,
    out_channels=dataset.num_classes,
    heads=8
).to(device)


# ============================================
# 4️⃣ Optimizer + Scheduler
# ============================================
optimizer = torch.optim.Adam(model.parameters(),
                             lr=0.005,
                             weight_decay=0.0005)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='max', factor=0.5, patience=20
)


# ============================================
# 5️⃣ Fonctions d'entraînement & d'évaluation
# ============================================
def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()


@torch.no_grad()
def test():
    model.eval()
    out = model(data.x, data.edge_index)
    pred = out.argmax(dim=1)
    accs = []
    for mask in [data.train_mask, data.val_mask, data.test_mask]:
        correct = pred[mask].eq(data.y[mask]).sum().item()
        acc = correct / mask.sum().item()
        accs.append(acc)
    return accs  # train_acc, val_acc, test_acc


# ============================================
# 6️⃣ Boucle d'entraînement avec Early Stopping
# ============================================
best_val_acc = 0
best_test_acc = 0
patience = 100
wait = 0
best_model = copy.deepcopy(model.state_dict())

for epoch in range(1, 501):
    loss = train()
    train_acc, val_acc, test_acc = test()

    # Scheduler update
    scheduler.step(val_acc)

    # Early stopping
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_test_acc = test_acc
        best_model = copy.deepcopy(model.state_dict())
        wait = 0
    else:
        wait += 1

    if wait >= patience:
        print(f"\n⛔ Early stopping à l'epoch {epoch}")
        break

    if epoch % 20 == 0:
        print(f"Epoch {epoch:03d} | Loss: {loss:.4f} | Train: {train_acc:.4f} | Val: {val_acc:.4f} | Test: {test_acc:.4f}")


# ============================================
# 7️⃣ Chargement du meilleur modèle
# ============================================
model.load_state_dict(best_model)
train_acc, val_acc, test_acc = test()
print("\n🎯 Performances finales:")
print(f"Train Acc: {train_acc:.4f}")
print(f"Validation Acc: {val_acc:.4f}")
print(f"Test Acc (meilleur modèle): ⭐ {test_acc:.4f} ⭐")


Device utilisé : cpu
Nombre de nœuds: 2708
Nombre de features par nœud: 1433
Nombre de classes: 7
Epoch 020 | Loss: 1.0812 | Train: 0.9643 | Val: 0.7960 | Test: 0.8160
Epoch 040 | Loss: 0.3633 | Train: 0.9929 | Val: 0.7960 | Test: 0.8110
Epoch 060 | Loss: 0.2070 | Train: 1.0000 | Val: 0.7880 | Test: 0.8120
Epoch 080 | Loss: 0.1567 | Train: 1.0000 | Val: 0.7880 | Test: 0.8070
Epoch 100 | Loss: 0.1609 | Train: 1.0000 | Val: 0.7880 | Test: 0.8030
Epoch 120 | Loss: 0.1279 | Train: 1.0000 | Val: 0.7920 | Test: 0.8010
Epoch 140 | Loss: 0.1856 | Train: 1.0000 | Val: 0.7900 | Test: 0.7990

⛔ Early stopping à l'epoch 142

🎯 Performances finales:
Train Acc: 0.9929
Validation Acc: 0.8040
Test Acc (meilleur modèle): ⭐ 0.8170 ⭐
